In [1]:
%pip install h2o

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 177.0 MB 131.3 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 91.5 MB/s eta 0:00:01
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=0533030546660b1c148809ab4577dcff647b0b881df5f2d4b3bf02c973cf12d2
  Stored in directory: /home/mk7516/.cache/pip/wheels/bf/d7/e0/905f059465fafaf07c9e79ba4c8d642ac646b01ea8c38b07d5
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=4f831c65e1c94dc6bcb6bd66006a00f5a1ba67ae0b30925cbe353bfd64aac98a
  Stored in directory: /home/mk7516/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built h2o future
You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_312-debug"; OpenJDK Runtime Environment (build 1.8.0_312-debug-b07); OpenJDK 64-Bit Server VM (build 25.312-b07-debug, mixed mode)
  Starting server from /home/mk7516/.local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /state/partition1/job-19141166/tmpnq_wvhl9
  JVM stdout: /state/partition1/job-19141166/tmpnq_wvhl9/h2o_mk7516_started_from_python.out
  JVM stderr: /state/partition1/job-19141166/tmpnq_wvhl9/h2o_mk7516_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_mk7516_ipig6o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.541 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [4]:
test, train = dataSetup('spambase')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
train.shape

(3450, 58)

In [6]:
x = train.columns
y = "class"
x.remove(y)

In [7]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%

12:59:15.280: GBM_lr_annealing_selection_AutoML_1_20220506_115942 [GBM lr_annealing] failed: water.exceptions.H2OIllegalArgumentException: Can only convert jobs producing a single Model or ModelContainer.

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_5_AutoML_1_20220506_115942

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.0022693557617032983
RMSE: 0.04763775563251588
LogLoss: 0.01415188989453129
Null degrees of freedom: 3449
Residual degrees of freedom: 3446
Null deviance: 4607.583535347977
Residual deviance: 97.64804027226594
AIC: 105.64804027226594
AUC: 0.9999830139964669
AUCPR: 0.9999893103346037
Gini: 0.9999660279929339

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6705786407462977: 


,,0,1,Error,Rate
0,0,1337.0,1.0,0.0007,(1.0/1338.0)
1,1,3.0,2109.0,0.0014,(3.0/2112.0)
2,Total,1340.0,2110.0,0.0012,(4.0/3450.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.670579,0.999053,202.0
1,max f2,0.670579,0.998769,202.0
2,max f0point5,0.716478,0.999526,199.0
3,max accuracy,0.670579,0.998841,202.0
4,max precision,0.999988,1.000000,0.0
5,max recall,0.242363,1.000000,220.0
6,max specificity,0.999988,1.000000,0.0
7,max absolute_mcc,0.670579,0.997560,202.0
8,max min_per_class_accuracy,0.670579,0.998580,202.0
9,max mean_per_class_accuracy,0.670579,0.998916,202.0



Gains/Lift Table: Avg response rate: 61.22 %, avg score: 61.20 %


...


Gains/Lift Table: Avg response rate: 61.22 %, avg score: 61.24 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.010145,0.999997,1.633523,1.633523,1.000000,0.999998,1.000000,0.999998,0.016572,0.016572,63.352273,63.352273,0.016572
1,2,0.020000,0.999995,1.633523,1.633523,1.000000,0.999996,1.000000,0.999997,0.016098,0.032670,63.352273,63.352273,0.032670
2,3,0.030145,0.999993,1.633523,1.633523,1.000000,0.999994,1.000000,0.999996,0.016572,0.049242,63.352273,63.352273,0.049242
3,4,0.040000,0.999991,1.633523,1.633523,1.000000,0.999991,1.000000,0.999995,0.016098,0.065341,63.352273,63.352273,0.065341
4,5,0.050435,0.999989,1.633523,1.633523,1.000000,0.999990,1.000000,0.999994,0.017045,0.082386,63.352273,63.352273,0.082386
5,6,0.100000,0.999977,1.633523,1.633523,1.000000,0.999983,1.000000,0.999989,0.080966,0.163352,63.352273,63.352273,0.163352
6,7,0.150145,0.999958,1.633523,1.633523,1.000000,0.999969,1.000000,0.999982,0.081913,0.245265,63.352273,63.352273,0.245265
7,8,0.200000,0.999926,1.633523,1.633523,1.000000,0.999944,1.000000,0.999973,0.081439,0.326705,63.352273,63.352273,0.326705
8,9,0.300000,0.999717,1.633523,1.633523,1.000000,0.999850,1.000000,0.999932,0.163352,0.490057,63.352273,63.352273,0.490057
9,10,0.400000,0.998857,1.633523,1.633523,1.000000,0.999400,1.000000,0.999799,0.163352,0.653409,63.352273,63.352273,0.653409




Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.984640,0.002911,0.986090,0.980741,0.983762,0.988571,0.984035
1,auc,0.998010,0.000429,0.998431,0.997410,0.998410,0.997820,0.997981
2,err,0.015360,0.002911,0.013910,0.019259,0.016238,0.011429,0.015965
3,err_count,10.600000,2.073644,9.000000,13.000000,12.000000,8.000000,11.000000
4,f0point5,0.985364,0.003737,0.984673,0.979553,0.987930,0.989242,0.985420
5,f1,0.987474,0.002529,0.989498,0.984468,0.986607,0.990632,0.986163
6,f2,0.989605,0.003692,0.994371,0.989433,0.985288,0.992026,0.986908
7,lift_top_group,1.634690,0.075911,1.522353,1.626506,1.645880,1.643193,1.735516
8,logloss,0.054003,0.003275,0.050994,0.058870,0.051735,0.052636,0.055777
9,max_per_class_error,0.026107,0.010277,0.036036,0.038462,0.017241,0.018248,0.020548



See the whole table with table.as_data_frame()


In [9]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_5_AutoML_1_20220506_115942,0.997917,0.0539822,0.998771,0.0184755,0.122252,0.0149455
StackedEnsemble_BestOfFamily_4_AutoML_1_20220506_115942,0.997898,0.0544929,0.998751,0.0200445,0.122493,0.0150044
GBM_grid_1_AutoML_1_20220506_115942_model_49,0.997842,0.0552731,0.998699,0.0222612,0.123302,0.0152035
GBM_grid_1_AutoML_1_20220506_115942_model_87,0.997706,0.0567282,0.998563,0.0195965,0.125007,0.0156268
GBM_grid_1_AutoML_1_20220506_115942_model_80,0.997635,0.0569868,0.998626,0.0214395,0.125095,0.0156487
GBM_grid_1_AutoML_1_20220506_115942_model_7,0.997588,0.0597448,0.998567,0.0217017,0.130117,0.0169304
GBM_grid_1_AutoML_1_20220506_115942_model_62,0.997569,0.0578831,0.998551,0.0194224,0.125827,0.0158325
GBM_grid_1_AutoML_1_20220506_115942_model_115,0.997568,0.0585749,0.99853,0.0195222,0.127437,0.0162402
GBM_grid_1_AutoML_1_20220506_115942_model_28,0.997551,0.0580458,0.998575,0.0219756,0.127128,0.0161617
GBM_grid_1_AutoML_1_20220506_115942_model_144,0.99755,0.0595152,0.998562,0.0224608,0.127083,0.0161502
